In [37]:
import json
import re
import pronto
import pickle
from tqdm import tqdm

In [2]:
flatten = lambda items: [i for sublist in items if sublist for i in sublist]

def get_CID(article: dict) -> str:
    if article.get('Record') and article.get('Record').get('RecordNumber'):
        return str(article.get('Record').get('RecordNumber'))
    else:
        return None

def get_ChEBI_from_wikidata(article: dict, wikidata_id: str) -> list:
    if not article['entities'][wikidata_id]['claims']:
        return []
    
    if 'P683' in set(article['entities'][wikidata_id]['claims'].keys()):
        return ["CHEBI:" + chebi_info['mainsnak']['datavalue']['value'] 
                for chebi_info in article['entities'][wikidata_id]['claims']['P683'] if chebi_info['mainsnak'].get('datavalue')
               ]
    else:
        return []
    
def get_synonym_from_wikidata(article: dict, wikidata_id: str) -> list:
    aliases = article['entities'][wikidata_id]['aliases']
    if len(aliases) is 0 or aliases.get('en') is None:
        return []
    
    return [synonyms['value'] for synonyms in aliases['en']]

def get_synonym_from_ChEBI(ont_entry):
    return [synonym.desc for synonym in list(ont_entry.synonyms)]

def get_synonym_from_PubChem(article):
    synonym_list = \
    [synonyms['StringValueList']
     for section in article['Record']['Section'] if section['TOCHeading'] == 'Names and Identifiers'
     for sub_section in section['Section'] if sub_section['TOCHeading'] == 'Synonyms'
     for synonym_info in sub_section['Section'] if synonym_info['TOCHeading'] != 'Removed Synonyms'
     for synonyms in synonym_info['Information']
    ]
    
    return flatten(synonym_list)

def reverse_table(table: dict):
    if not isinstance(list(table.values())[0], list):
        return {v: k for k, v in table.items()}
    
    new_table = {}
    for k, values in table.items():
        for v in values:
            if new_table.get(v):
                new_table[v] += [k]
            else:
                new_table[v] = [k]
                
    return new_table

In [3]:
# PubChem記事からChEBI-IDをスクレイピング

cid2chebi_table = {}
no_ChEBI_CID_list = []
with open("../../data/pubchem_articles.jsonl", 'r') as f:
    for line in f:
        article = json.loads(line)
        cid = get_CID(article)
        
        if cid is None:
            continue
        
        chebi_id = list(set(re.findall(r'CHEBI:[\d]+', line)))
        
        if len(chebi_id) is 0:
            no_ChEBI_CID_list.append(cid)
        
        cid2chebi_table[cid] = chebi_id
        
CID2ChEBI_links = len([k for k, v in cid2chebi_table.items() if v])
print("ChEBI-IDが紐付けられたCID:", CID2ChEBI_links)

ChEBI-IDが紐付けられたCID: 2904


In [4]:
# WikiDataからChEBI-IDをスクレイピング

wd2chebi_table = {}
no_ChEBI_WDID_list =[]
with open("../../data/compound_wikidata.jsonl", 'r') as f:
    for line in f:
        wikidata_id = re.findall(r'^\{\"entities\": {\"(Q[\d]+)\"', line)[0]
        
        if not wikidata_id:
            continue
        
        article = json.loads(line)
        chebi_id = get_ChEBI_from_wikidata(article, wikidata_id)
        
        if len(chebi_id) is 0:
            no_ChEBI_WDID_list.append(wikidata_id)
        
        wd2chebi_table[wikidata_id] = chebi_id

wd2chebi_links = len([k for k, v in wd2chebi_table.items() if v])
print("ChEBI-IDが紐付けられたWikiDataID:", wd2chebi_links)

ChEBI-IDが紐付けられたWikiDataID: 3159


In [5]:
with open("../../data/ENE_compound_name.txt", 'r', encoding='utf-8') as f:
    pageid_list = [str(json.loads(entry)['pageid']) for entry in f.readlines()]    

with open("../../data/pageID2CID_using_wikidata.json", 'r') as f:
    pageid2cid_table = json.load(f)

with open("../../data/pageID2wikidataID.json", 'r') as f:
    pageid2wd_table = json.load(f)

In [7]:
# make reverse table
cid2pageid_table = reverse_table(pageid2cid_table)
wd2pageid_table = reverse_table(pageid2wd_table)
chebi2pageid_table = reverse_table(pageid2chebi_table)

In [6]:
# Page_id と ChEBI-ID の紐づけ

pageid2chebi_table = {}
no_ChEBI_pageid_list = []
for page_id in pageid_list:
    ChEBI_list = []
    
    if pageid2cid_table.get(page_id):
        cid_list = pageid2cid_table[page_id]
        ChEBI_list += flatten([cid2chebi_table[cid] for cid in cid_list])
    
    if pageid2wd_table.get(page_id):
        wikidata_id = pageid2wd_table[page_id]
        if wd2chebi_table.get(wikidata_id):
            ChEBI_list += wd2chebi_table[wikidata_id]
        
    ChEBI_list = list(set(ChEBI_list))
    
    if len(ChEBI_list) is 0:
        no_ChEBI_pageid_list.append(page_id)
    
    pageid2chebi_table[page_id] = ChEBI_list

pageid2ChEBI_links = len([k for k, v in pageid2chebi_table.items() if v])
print("ChEBI-IDが紐付けられたWikipedia記事:", pageid2ChEBI_links)

ChEBI-IDが紐付けられたWikipedia記事: 3294


In [8]:
synonyms_dict = {str(page_id): [] for page_id in pageid_list}

with open("../../data/pubchem_articles.jsonl", 'r') as f:
    for line in f:
        article = json.loads(line)
        cid = get_CID(article)
        
        if cid is None:
            continue
        
        synonyms = get_synonym_from_PubChem(article)
        for page_id in cid2pageid_table[cid]:
            synonyms_dict[page_id] += synonyms
            
            
with open("../../data/compound_wikidata.jsonl", 'r') as f:
    for line in f:
        wikidata_id = re.findall(r'^\{\"entities\": {\"(Q[\d]+)\"', line)[0]
        
        if not wikidata_id:
            continue
            
        page_id = wd2pageid_table.get(wikidata_id)
        if page_id is None:
            continue
        
        article = json.loads(line)
        synonyms = get_synonym_from_wikidata(article, wikidata_id)
        synonyms_dict[page_id] += synonyms

## 同義語情報を用いて Linking

In [9]:
ont = pronto.Ontology("../../data/chebi.owl")

In [15]:
def is_synonym(s1, s2):
    return set(s1) & set(s2)

In [30]:
chebi_nonlink_pageid = [page_id for page_id, chebi_ids in pageid2chebi_table.items() if len(chebi_ids) is 0]
print(len(chebi_nonlink_pageid))

2526


In [35]:
new_pageid2chebi_table = pageid2chebi_table.copy()
count = 0
for ChEBI_entry in tqdm(ont):
    ChEBI_synonyms = get_synonym_from_ChEBI(ChEBI_entry)
    for page_id in chebi_nonlink_pageid:
        synonyms = synonyms_dict[page_id]
        if is_synonym(ChEBI_synonyms, synonyms):
            count += 1
            new_pageid2chebi_table[page_id] = list(set(new_pageid2chebi_table[page_id] + [ChEBI_entry.id]))
            
print("New ChEBI-IDs:", count)

100%|██████████| 130627/130627 [09:24<00:00, 231.33it/s]

New ChEBI-IDs: 135


In [36]:
pageid2ChEBI_links = len([k for k, v in new_pageid2chebi_table.items() if v])
print("ChEBI-IDが紐付けられたWikipedia記事:", pageid2ChEBI_links)

ChEBI-IDが紐付けられたWikipedia記事: 3397


In [38]:
with open("../../data/pageid2ChEBI.json", 'w') as f:
    json.dump(new_pageid2chebi_table, f)

In [42]:
with open("../../data/ENE_compound_name.txt", 'r', encoding='utf-8') as f:
    title = [str(json.loads(entry)['title']) for entry in f.readlines() if str(json.loads(entry)['pageid']) in chebi_nonlink_pageid]    

In [43]:
title

['カリウムヘキサメチルジシラジド',
 'セメンタイト',
 '超酸化ナトリウム',
 '塩化ウラン(VI)',
 'メタプロスカリン',
 'トリペプチド',
 '低次フラーレン',
 '安息香酸カリウム',
 'ナラシン',
 '炭酸銀(I)',
 '臭素酸銀(I)',
 'クロトン酸エチル',
 'ウリジン二リン酸ガラクトース',
 '酪酸ヘプチル',
 '亜硫酸マグネシウム',
 'ポリビニルポリピロリドン',
 'テトラフェニルメタン',
 'バラトール',
 '炭化カルシウム',
 '3-ベンズヒドリルモルホリン',
 'オイルシェールガス',
 'フッ化銅(II)',
 '三酸化テルル',
 'ヘキサオキソトリシクロブタベンゼン',
 'ベンザイン',
 'フェリクローム',
 'ヘプテン',
 'ネブラセタム',
 'フッ化バナジウム(IV)',
 'フラボガロン酸',
 'マンノビオース',
 '発煙硝酸',
 'ルビセン',
 'トリス(ジベンジリデンアセトン)ジパラジウム(0)',
 'ゴシポール',
 '塩化ウラン(IV)',
 'ペンタノナコンタン',
 '硝酸ストロンチウム',
 '硫化亜鉛',
 'ヤンゴニン',
 '三フッ化プルトニウム',
 '1-ブロモヘキサン',
 'ハベナリオール',
 '六フッ化ルテニウム',
 'コリン',
 'ヨウ化ガリウム(III)',
 'トリ硫酸',
 '塩化リン',
 'ジチオカルボン酸',
 'ヘルバセチン',
 '酸化タングステン(III)',
 'ブロモトリフルオロメタン',
 '3-メチルチオプロピオン酸エチル',
 'テルフェニル',
 'アルキルグリセロール',
 '2-メチル-6-ニトロ安息香酸無水物',
 'スルフィン酸',
 '過テクネチウム酸塩',
 '炭化タンタルハフニウム',
 'クロラニル酸',
 '臭化ニトロシル',
 'テロメスタチン',
 'イサト酸',
 'スパンデックス',
 'チオシアン酸エチル',
 '硫化水素カリウム',
 'トラボプロスト',
 'シクロペンタジエニルナトリウム',
 'ヨウ化トリウム(IV)',
 'アイオノマー',
 'DHMO',
 'モモルデノール',
 '2-エチル-3-メチルピラジン',
 'ウステキヌマブ',
 'キセノン